In [1]:
import pandas as pd
import numpy as np
from datetime import time as dtime
from time import perf_counter
import math

In [2]:
# USER CONFIGURABLE SETTINGS:

CSV_PATH = "../data/Options_data_2023.csv"
ENTRY_TIME     = dtime(9, 20)
EXIT_TIME      = dtime(15, 20)
STOPLOSS_PCT   = 0.50
LOT_SIZE       = 15
BASE_NAV       = 100.0
START_CAPITAL  = 1000.0  # for "Available capital" column; equity curve is based on NAV=100
REENTRY        = False   # set True if you want re-entry after SL (same minute)
OUTPUT_XLSX    = "BN_Strangle_Backtest.xlsx"
TARGET_PREMIUM = 50.0


In [3]:
def data_preprocessing(csv_path):
    data = pd.read_csv(csv_path, index_col = 0,
                       engine = "pyarrow",
                       dtype = {"Call/Put": "category",
                                "Ticker": "category"
                                })

    # date dtypes
    data["Date"] = pd.to_datetime(data["Date"], format = "%Y-%m-%d").dt.normalize()
    data["Time"] = pd.to_datetime(data["Time"].astype(str), format="%H:%M:%S", errors="coerce")
    data["Time"] = data["Time"].dt.time
    data["Ticker"] = data["Ticker"].astype(str)

    strike = data["Ticker"].str.extract(r"(\d+)").astype(float)
    data["Strike"] = strike[0]

    data["minute"] = pd.to_datetime(data["Date"].astype(str) + " " + pd.Series(data["Time"].astype(str)))

    return data
    


In [4]:
data = data_preprocessing(CSV_PATH)

In [ ]:
data

,Date,Ticker,Time,Open,High,Low,Close,Call/Put,Strike,minute
,,,,,,,,,,
0,2023-01-02,BANKNIFTY37000PE,09:15:59,3.40,3.40,2.85,3.00,PE,37000.0,2023-01-02 09:15:59
1,2023-01-02,BANKNIFTY43800CE,09:15:59,79.85,104.25,73.90,91.20,CE,43800.0,2023-01-02 09:15:59
2,2023-01-02,BANKNIFTY43900PE,09:15:59,867.95,883.15,826.75,826.75,PE,43900.0,2023-01-02 09:15:59
3,2023-01-02,BANKNIFTY44000PE,09:15:59,957.25,981.05,911.00,914.70,PE,44000.0,2023-01-02 09:15:59
4,2023-01-02,BANKNIFTY44200CE,09:15:59,37.90,42.95,27.35,37.10,CE,44200.0,2023-01-02 09:15:59
...,...,...,...,...,...,...,...,...,...,...
10266676,2024-01-02,BANKNIFTY47000CE,15:29:59,615.00,615.95,612.15,614.95,CE,47000.0,2024-01-02 15:29:59
10266677,2024-01-02,BANKNIFTY47000PE,15:29:59,11.95,12.45,11.65,12.25,PE,47000.0,2024-01-02 15:29:59
10266678,2024-01-02,BANKNIFTY47500PE,15:29:59,106.45,109.50,105.10,107.65,PE,47500.0,2024-01-02 15:29:59


: 